<a href="https://colab.research.google.com/github/damiancyrana/colab-notebooks/blob/main/best_prices_currency_exchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kneed
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.graph_objs as go
from kneed import KneeLocator
from matplotlib import colormaps
import matplotlib.pyplot as plt


In [36]:
# Załadowanie danych
file_path = '/content/drive/MyDrive/Colab Notebooks/Dane historyczne walut od 01.01.2002/EURPLN_D1.csv'
data = pd.read_csv(file_path)
data['ctmString'] = pd.to_datetime(data['ctmString'])
data.set_index('ctmString', inplace=True)
data = data.iloc[::-1].copy()


# Funkcja do obliczania RSI
def calculate_rsi(data, column='close', period=14):
    delta = data[column].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


# Metoda łokcia do wyboru liczby skupień K dla k-średnich
def elbow_method(data, max_k=10):
    inertias = []
    for k in range(1, max_k+1):
        kmeans = KMeans(n_clusters=k, random_state=42).fit(data)
        inertias.append(kmeans.inertia_)
    kn = KneeLocator(range(1, max_k+1), inertias, curve='convex', direction='decreasing')
    return kn.elbow


# Obliczanie RSI
data['RSI'] = calculate_rsi(data, 'close')

# Normalizacja danych cenowych przed zastosowaniem k-średnich
scaler = StandardScaler()
data['close_scaled'] = scaler.fit_transform(data['close'].values.reshape(-1, 1))

# Użycie metody łokcia do określenia optymalnej liczby skupień
optimal_k = elbow_method(data[['close_scaled']], max_k=10)

# Wykonanie k-średnich z optymalną liczbą klastrów
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
data.loc[:, 'Cluster'] = kmeans.fit_predict(data[['close_scaled']])

# Znalezienie średnich cen dla każdego klastra
cluster_avg_prices = data.groupby('Cluster')['close'].mean().sort_values()

# Przyporządkowanie kolorów do klastrów
colors = colormaps['viridis'](np.linspace(0, 1, optimal_k))
cluster_colors = {cluster: f'rgba({colors[i][0]*255}, {colors[i][1]*255}, {colors[i][2]*255}, 0.6)' for i, cluster in enumerate(cluster_avg_prices.index)}

# Ustawienie zielonego koloru dla najlepszego klastra i czerwonego dla najgorszego
cluster_colors[cluster_avg_prices.idxmin()] = 'rgba(69, 255, 30, 0.6)'
cluster_colors[cluster_avg_prices.idxmax()] = 'rgba(255, 99, 71, 0.6)'

# Tworzenie wykresów
fig_price = go.Figure()

# Dodawanie wykresu cen zamknięcia z klastrami cenowymi oraz wypełnienie kolorami
for cluster_label in data['Cluster'].unique():
    cluster_data = data[data['Cluster'] == cluster_label]
    color = cluster_colors[cluster_label]
    fig_price.add_trace(go.Scatter(x=cluster_data.index, y=cluster_data['close'], mode='markers', name=f'Cluster {cluster_label}', marker=dict(color=color)))

    # Zaznaczenie granic każdego klastra
    min_price, max_price = cluster_data['close'].min(), cluster_data['close'].max()
    fig_price.add_hrect(y0=min_price, y1=max_price, line_width=0, fillcolor=color, opacity=0.5)

    # Dodawanie linii rozdzielających klastry
    #fig_price.add_hline(y=min_price, line_dash="dash", line_color="grey", annotation_text=f"Min: {min_price:.2f}")
    fig_price.add_hline(y=max_price, line_dash="dash", line_color="grey", annotation_text=f"Max: {max_price:.2f}")

# Aktualizacja układu wykresu
fig_price.update_layout(title='Klastry Cenowe z RSI', xaxis_title='Data', yaxis_title='Cena Zamknięcia')
fig_price.show()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang